# Iteration Workflow

If you are updating a network diagram based on changes made from an earlier session, use this workflow.

If you are looking at a network for the first time, you should try the [Exploratory Workflow](./exploratory_workflow.ipynb).

## To create an updated structure after making new commits:

* Generate an updated edge list.
* Copy the previous position file to use as a starting point for the next visualization.
* Open Cytoscape or destroy existing collections if Cytoscape is already running.
* Run all cells to load the visualization.
* [Save the new layout if you make changes](http://localhost:8888/notebooks/lein-topology.ipynb#Save-the-updated-layout-coordinates-if-you-make-changes).

In [110]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.style import StyleUtil

import py2cytoscape.util.cytoscapejs as cyjs
import py2cytoscape.cytoscapejs as renderer

import igraph as igraph
import pandas as pd
import numpy as np

import sand.io as io
import sand.graph as sg
import sand.cytoscape.positions as scp
import sand.cytoscape.themes as themes
import sand.cytoscape.version as version

## Verify that Cytoscape is running and get the current version of Cytoscape.

In [111]:
version.print_version()

{
  "cytoscapeVersion": "3.4.0", 
  "apiVersion": "v1"
}


## Load the dataset as a weighted edge list

In [112]:
# Change this path to the directory containing your dataset: This example will use the output of lein-topology:
%cd "~/dev/lein-topology"

/Users/bnorton/dev/lein-topology


In [113]:
network_collection = "lein-topology"
network_name = "57af741"

data_path = "./data/" + network_collection + "-" + network_name

In [114]:
edge_file = data_path + ".csv"
positions_file = data_path + "-positions.csv"
image_file = data_path + ".svg"

In [115]:
edgelist = io.csv_to_edgelist(edge_file)
g = sg.edgelist_to_igraph(edgelist)

In [116]:
g.summary()

'IGRAPH D-W- 107 206 -- \n+ attr: label (v), weight (e)'

## Extract the subgraph of local namespaces 

There are some analyses where it will be useful to see all the vertices. For the high-level architecture diagram, we can focus on the functions local to the library's namespaces. We'll also keep functions used for output.

In [117]:
# List all patterns of vertex names that we want to keep:
names_to_keep = ['topology', 'clojure.core/*err*', 'clojure.core/println']

In [118]:
lv = g.vs(lambda v: any(match in v['label'] for match in names_to_keep))
lg = g.subgraph(lv)

In [119]:
lg.summary()

'IGRAPH D-W- 26 26 -- \n+ attr: label (v), weight (e)'

## Load into Cytoscape with a default layout

In [120]:
# Create py2cytoscape client
cy = CyRestClient()

In [121]:
# Load the network
network = cy.network.create_from_igraph(lg, name=network_name, collection=network_collection)

## Customize the style

Use one of the included themes, or build your own.

In [122]:
style = cy.style.create('Ops')
style.update_defaults(themes.ops.settings)

# Map the label property in the igraph data to Cytoscape's NODE_LABEL visual property
style.create_passthrough_mapping(column='label', vp='NODE_LABEL', col_type='String')

cy.style.apply(style, network)

## Layout

In [123]:
# Apply default layout
cy.layout.apply(name='force-directed', network=network)

In [124]:
# Load layout coordinates from a previous session
scp.layout_from_positions_csv(network, positions_file, cy)

## Save the updated layout coordinates if you make changes

One benefit of this workflow is the ability to manually tweak the algorithmic network layout in Cytoscape.

After making changes, save the coordinates for a later session:

In [81]:
scp.positions_to_csv(network=network, path=positions_file)

## Generate an SVG export

Position the network in Cytoscape the way you want it, then trigger this export:

In [84]:
from subprocess import call

svg_url = "http://localhost:1234/v1/networks/" + str(network.get_id()) + "/views/first.svg"
call(["wget", svg_url, "-O", image_file])

0

In [85]:
print(svg_url)

http://localhost:1234/v1/networks/960/views/first.svg
